In [1]:
# 스파크 세션 생성
import pyspark
from pyspark.sql import SparkSession

# SynapseML : 마이크로소프트에서 만든 스파크용 ML 라이브러리
# https://microsoft.github.io/SynapseML/
# config에서 synapseml 모듈 불러오기
spark = SparkSession.builder \
    .appName("Spark study - 240104") \
    .getOrCreate()

131072x1 화면 크기가 잘못됐습니다. 문제가 예상됩니다
24/01/05 02:23:49 WARN Utils: Your hostname, KJH-DESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.69.220 instead (on interface eth0)
24/01/05 02:23:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 02:23:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/05 02:23:51 WARN RapidsPluginUtils: RAPIDS Accelerator 23.12.0 using cudf 23.12.0.
24/01/05 02:23:51 WARN RapidsPluginUtils: spark.rapids.sql.multiThreadedRead.numThreads is set to 20.
24/01/05 02:23:51 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
24/01/05 02:23:51 WARN RapidsPluginUtils: spark.rapids.sql.explain is set to `NOT_ON_GPU`. Set it to 'NONE' to suppress the diag

In [2]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

import torch
import torch.nn as nn

class FieldAwareFactorizationMachine(nn.Module):
    def __init__(self, field_dims, embed_dim):
        super(FieldAwareFactorizationMachine, self).__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(field_dim, embed_dim) for field_dim in field_dims
        ])
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        embeddings = [embedding(x[:, i]) for i, embedding in enumerate(self.embeddings)]
        interactions = torch.sum(torch.stack(embeddings), dim=0)
        embeddings_squared = torch.stack([embedding**2 for embedding in embeddings])
        ffm_term = torch.sum(interactions**2 - torch.sum(embeddings_squared, dim=0), dim=1, keepdim=True)
        return ffm_term + self.bias

def torch_model_udf(model):
    def udf_func(*args):
        with torch.no_grad():
            input_tensor = torch.tensor([args], dtype=torch.long)
            output_tensor = model(input_tensor)
            return float(output_tensor.numpy().item())

    return udf(udf_func, DoubleType())

data = [(1, 2, 3, 4, 5.0), (2, 3, 4, 5, 6.0), (3, 4, 5, 6, 7.0)]
columns = ["field1", "field2", "field3", "field4", "label"]
df = spark.createDataFrame(data, columns)

field_dims = [5, 6, 7, 8] 
embed_dim = 5
ffm_model = FieldAwareFactorizationMachine(field_dims, embed_dim)
udf_model = torch_model_udf(ffm_model)

result_df = df.withColumn("prediction", udf_model(*columns[:-1]))
result_df.show()

24/01/05 02:23:53 WARN GpuOverrides: 
!Exec <CollectLimitExec> cannot run on GPU because the Exec CollectLimitExec has been disabled, and is disabled by default because Collect Limit replacement can be slower on the GPU, if huge number of rows in a batch it could help by limiting the number of rows transferred from GPU to CPU. Set spark.rapids.sql.exec.CollectLimitExec to true if you wish to enable it
  @Partitioning <SinglePartition$> could run on GPU
    ! <BatchEvalPythonExec> cannot run on GPU because GPU does not currently support the operator class org.apache.spark.sql.execution.python.BatchEvalPythonExec
      @Expression <PythonUDF> udf_func(field1#0L, field2#1L, field3#2L, field4#3L)#10 could not block GPU acceleration
        @Expression <AttributeReference> field1#0L could run on GPU
        @Expression <AttributeReference> field2#1L could run on GPU
        @Expression <AttributeReference> field3#2L could run on GPU
        @Expression <AttributeReference> field4#3L could r

+------+------+------+------+-----+------------+
|field1|field2|field3|field4|label|  prediction|
+------+------+------+------+-----+------------+
|     1|     2|     3|     4|  5.0|-0.513734043|
|     2|     3|     4|     5|  6.0|-9.016757011|
|     3|     4|     5|     6|  7.0|-7.097997665|
+------+------+------+------+-----+------------+

